## BERT ANALYSIS

LIBRARIES LOADING

In [28]:
import pandas as pd
import numpy as np
import logging
import torch
from datasets import Dataset, DatasetDict
from transformers import DistilBertModel
from transformers import AutoTokenizer, DistilBertTokenizer
from torch.utils.data import Dataset, DataLoader
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

This approach will be very useful not only for going step by step with the classification but also for allowing to decide the trade-off of accuracy and digits returned.

DATA PREPROCESSING

In [29]:
data_path = '../data/coverwallet.xlsx'
df = pd.read_excel(data_path)
df = df.dropna()
def truncate_naics_and_prepare_data(df, column_name, num_digits):
    """
    Truncates the NAICS codes in the specified column to the desired number of digits.

    :param df: pandas DataFrame containing the NAICS codes.
    :param column_name: the name of the column with the NAICS codes.
    :param num_digits: the number of digits to truncate to.
    :return: A copy of the DataFrame with the NAICS codes truncated.
    """
    # Validate the number of digits
    if not isinstance(num_digits, int) or num_digits <= 0:
        logging.error("Number of digits must be a positive integer")
        raise ValueError("Number of digits must be a positive integer")
    
    # Make a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()
    def truncate_code(code):
        """
        Truncates or pads the NAICS code to the specified number of digits.
        :param code: the NAICS code to be truncated.
        :return: The truncated or original NAICS code as a string.
        """
        try:
            # Ensure the code is a string
            code_str = str(code)
            # Truncate the code if it's longer than num_digits
            return code_str[:num_digits].ljust(num_digits, '0')
        except Exception as e:
            logging.exception("Error truncating code: {}".format(code))
            return code
        
    # Apply the truncation function to the specified column
    df_copy[column_name] = df_copy[column_name].apply(truncate_code)
    # Try to convert the truncated column to integers
    try:
        df_copy[column_name] = df_copy[column_name].astype(int)
    except ValueError as e:
        logging.warning("Could not convert truncated codes to integers: {}".format(e))
        # Keep the column as strings if conversion fails
        pass
    
    labels = df_copy['NAICS'].unique().tolist()
    id2label = {idx: label for idx, label in enumerate(labels)}
    label2id = {label: idx for idx, label in enumerate(labels)}
    df_copy['label'] = df_copy['NAICS'].map(label2id)
    logging.info("NAICS codes processed successfully. Here's the head of the processed DataFrame:")
    logging.info("\n%s", df_copy.head())
    df_copy_train, df_copy_final_val = train_test_split(df_copy, test_size=0.15, shuffle=True, random_state=42)
    
    dataset_train = Dataset.from_pandas(df_copy_train)
    dataset_final_val = Dataset.from_pandas(df_copy_final_val)
    
    return df_copy,  dataset_train, dataset_final_val
'''
# Configuration k-fold
    num_folds = 3
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    kfold_datasets = []

    for fold, (train_indices, val_indices) in enumerate(kf.split(dataset_train)):
        train_dataset = dataset_train.select(train_indices)
        val_dataset = dataset_train.select(val_indices)
        
        dataset_dict = {
            'train': train_dataset,
            'validation': val_dataset
        }

        features_dict = {
            "NAICS": dataset_train["NAICS"],
            "BUSINESS_DESCRIPTION": dataset_train["BUSINESS_DESCRIPTION"],
        }
    
        kfold_datasets.append(dataset_dict)
        logging.info(f"Processed fold {fold + 1}")

    for i, dataset_dict in enumerate(kfold_datasets):
        for split in dataset_dict.keys():
            dataset_dict[split] = dataset_dict[split].map(lambda example: {key: example[key] for key in features_dict.keys()})

        logging.info(f"DatasetDict for Fold {i + 1}:")
        for split, dataset in dataset_dict.items():
            logging.info(f"  {split} split: {dataset}")
            
    logging.info("NAICS codes truncated successfully. Here's the head of the truncated DataFrame:")
    logging.info("\n%s", df_copy.head())
    logging.info("Number of unique NAICS labels: %d", len(labels))
    '''
    #return df_copy, kfold_datasets, dataset_train, dataset_final_val

'\n# Configuration k-fold\n    num_folds = 3\n    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)\n    kfold_datasets = []\n\n    for fold, (train_indices, val_indices) in enumerate(kf.split(dataset_train)):\n        train_dataset = dataset_train.select(train_indices)\n        val_dataset = dataset_train.select(val_indices)\n        \n        dataset_dict = {\n            \'train\': train_dataset,\n            \'validation\': val_dataset\n        }\n\n        features_dict = {\n            "NAICS": dataset_train["NAICS"],\n            "BUSINESS_DESCRIPTION": dataset_train["BUSINESS_DESCRIPTION"],\n        }\n    \n        kfold_datasets.append(dataset_dict)\n        logging.info(f"Processed fold {fold + 1}")\n\n    for i, dataset_dict in enumerate(kfold_datasets):\n        for split in dataset_dict.keys():\n            dataset_dict[split] = dataset_dict[split].map(lambda example: {key: example[key] for key in features_dict.keys()})\n\n        logging.info(f"DatasetDict fo

In [35]:
#df_2_digits, kfold_2_digits, dataset_train_2_digits, dataset_final_val_2_digits = truncate_naics_and_prepare_data(df, 'NAICS', 2)
df_2_digits, dataset_train_2_digits, dataset_final_val_2_digits = truncate_naics_and_prepare_data(df, 'NAICS', 2)
# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 3
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
print(df_2_digits.columns)
print(sorted(df_2_digits['label'].unique()))


INFO: NAICS codes processed successfully. Here's the head of the processed DataFrame:
INFO: 
   NAICS                               BUSINESS_DESCRIPTION  label
0     72  Zenyai Viet Cajun & Pho Restaurant is dedicate...      0
1     54  Kilduff Underground Engineering, Inc. (KUE) is...      1
2     45  024™ is a premium home fragrance brand that de...      2
3     56  Our Services include Office Cleaning Carpet cl...      3
4     62                    NYS Licensed Home Health Agency      4


Index(['NAICS', 'BUSINESS_DESCRIPTION', 'label'], dtype='object')
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [37]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['BUSINESS_DESCRIPTION'].values
        self.targets = dataframe['label'].values
        

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        text = str(self.text[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )
        return {
            'ids': inputs['input_ids'].flatten(),
            'mask': inputs['attention_mask'].flatten(),
            'targets': torch.tensor(self.targets[idx], dtype=torch.float)
            #'targets': self.targets[idx]
        }

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, new_data):
        """
        Setter para el atributo 'data'. Este método permite cambiar el valor de 'data'.
        """
        self._data = new_data
        # Actualizar 'text' y 'targets' si es necesario
        self.text = new_data['BUSINESS_DESCRIPTION']
        self.targets = list(new_data['label'])

In [50]:
train_size = 0.8
train_data=df_2_digits.sample(frac=train_size,random_state=200)
test_data=df_2_digits.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(df_2_digits.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = CustomDataset(train_data, tokenizer, MAX_LEN)
print(training_set.__getitem__(2))
testing_set = CustomDataset(test_data, tokenizer, MAX_LEN)
print(f"Longitud del training_set: {len(training_set)}")

# Intenta acceder a un elemento para ver si está correctamente configurado
try:
    sample = training_set[0]
    print(f"Primer elemento del training_set: {sample}")
except Exception as e:
    print(f"Error al acceder al training_set: {e}")


train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

#training_loader = DataLoader(training_set, **train_params)
def custom_collate_fn(batch):
    # Aquí puedes definir cómo transformar los datos de `batch`
    # Por ejemplo, agrupar `input_ids`, `attention_mask` y `labels` de manera personalizada
    return {
        'input_ids': torch.stack([item['input_ids'] for item in batch]),
        'attention_mask': torch.stack([item['attention_mask'] for item in batch]),
        'labels': torch.tensor([item['labels'] for item in batch])
    }

training_loader = DataLoader(dataset=training_set, batch_size=2, shuffle=True, num_workers=4)
# Supongamos que 'data_loader' es tu DataLoader
data_loader_iter = iter(training_loader)

# Obtener el siguiente lote de datos
try:
    batch = next(data_loader_iter)
except StopIteration:
    # Esto captura el caso cuando el iterador no tiene más elementos.
    pass


#training_loader = DataLoader(training_set, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=0)
#print(training_loader)
testing_loader = DataLoader(testing_set, **test_params)

'''
try:
    for i, batch in enumerate(training_loader):
        print(f"Batch {i}: {batch}")
        if i == 1:  # Imprime solo los primeros dos batches para evitar sobrecargar la salida
            break
except Exception as e:
    print(f"Error al iterar sobre training_loader: {e}")

'''




FULL Dataset: (14175, 3)
TRAIN Dataset: (11340, 3)
TEST Dataset: (2835, 3)
{'ids': tensor([  101,  2909,  2072, 18558, 19454, 13700,  2015,  4297,  1010,  2003,
         2019,  2009, 10552,  1998,  7661,  3274,  4730,  2578,  2194,  1012,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,   

IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/edu/.cache/pypoetry/virtualenvs/zrive-ds-J_lydEot-py3.11/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/edu/.cache/pypoetry/virtualenvs/zrive-ds-J_lydEot-py3.11/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/edu/.cache/pypoetry/virtualenvs/zrive-ds-J_lydEot-py3.11/lib/python3.11/site-packages/datasets/arrow_dataset.py", line 2816, in __getitems__
    return [{col: array[i] for col, array in batch.items()} for i in range(n_examples)]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/edu/.cache/pypoetry/virtualenvs/zrive-ds-J_lydEo

In [14]:

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 24)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [15]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [20]:
from tqdm import tqdm
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    # Verificar si el DataLoader tiene batches para procesar
    for i, batch in enumerate(training_loader):
        print(f"Batch {i}: {batch}")
        break  # Romper después del primer batch para no imprimir todo si hay datos
    '''
    for batch_idx, batch in enumerate(tqdm(training_loader, desc=f"Epoch {epoch+1}")):
        input_ids = batch['ids'].to(device)
        attention_mask = batch['mask'].to(device)
        labels = batch['targets'].to(device).unsqueeze(1)  # Asegúrate de que las etiquetas tengan las dimensiones correctas

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)

        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Average loss in epoch {epoch+1}: {total_loss / len(training_loader)}")
    '''
    
    
    '''
    for _ , data in tqdm(enumerate(training_loader), desc=f"Epoch {epoch}"):
        print("hola")
    
        input_ids = data['input_ids'].to(device, dtype=torch.long)
        attention_mask = data['attention_mask'].to(device, dtype=torch.long)
        targets = data['labels'].to(device, dtype=torch.float)  # Ajustado para usar 'labels'

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
    '''

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [24]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]


TypeError: Index must be an integer

## - - - - - - - - - - - - 











MODEL

In [12]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 128 
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1
EPOCHS = 2
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [13]:
class NAICSDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        """
        Inicializa el dataset.

        :param dataframe: DataFrame de pandas que contiene las descripciones de negocio y sus etiquetas.
        :param tokenizer: Tokenizador de transformers utilizado para procesar el texto.
        :param max_len: Longitud máxima de la secuencia de tokens.
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['BUSINESS_DESCRIPTION']  
        self.targets = list(dataframe['label'])  
        self.max_len = max_len

    def __len__(self):
        """
        Devuelve la cantidad de elementos en el dataset.
        """
        return len(self.text)

    def __getitem__(self, index):
        """
        Obtiene el ítem en el índice especificado con tokenización y preparación para el modelo.
        """
    # Manejar el caso inusual de que index sea una lista (debido al error mencionado)
        if isinstance(index, list):
        # Esta situación no debería ocurrir con el uso estándar de PyTorch DataLoader
        # Se puede lanzar un error, registrar un mensaje o manejar de otra manera según sea necesario
            raise ValueError("Index is a list, expected a single integer value.")
    
        # Continuar con el procesamiento normal cuando index es un entero
        text = str(self.text[index])  # Asume que self.text es indexable con enteros
        text = " ".join(text.split())

        # Codificar el texto
        inputs = self.tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=self.max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=False  # DistilBERT no utiliza token_type_ids
    )

    # Preparar la salida
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)  # Asegúrate de que esto coincide con el formato esperado por tu modelo
        }
     
    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, new_data):
        """
        Setter para el atributo 'data'. Este método permite cambiar el valor de 'data'.
        """
        self._data = new_data
        # Actualizar 'text' y 'targets' si es necesario
        self.text = new_data['BUSINESS_DESCRIPTION']
        self.targets = list(new_data['label'])


In [14]:
def create_data_loaders_from_datasets(train_dataset, valid_dataset, tokenizer, max_len, train_batch_size, valid_batch_size):
    """
    Crea DataLoader para los conjuntos de datos de entrenamiento y validación.

    :param train_dataset: DataFrame de entrenamiento o NAICSDataset ya inicializado.
    :param valid_dataset: DataFrame de validación o NAICSDataset ya inicializado.
    :param tokenizer: Tokenizador de transformers utilizado para procesar el texto.
    :param max_len: Longitud máxima de la secuencia de tokens.
    :param train_batch_size: Tamaño del lote para el entrenamiento.
    :param valid_batch_size: Tamaño del lote para la validación.
    :return: Tuple de DataLoaders para entrenamiento y validación.
    """
    # Crear DataLoader para el conjunto de entrenamiento
    train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
    
    # Crear DataLoader para el conjunto de validación
    valid_loader = DataLoader(valid_dataset, batch_size=valid_batch_size, shuffle=False)
    
    return train_loader, valid_loader

def prepare_dataloaders(kfold_datasets, tokenizer, max_len, train_batch_size, valid_batch_size):
    """
    Esta función asume que kfold_datasets contiene pares de DataFrames de pandas para entrenamiento y validación en cada fold.
    """
    train_dataloaders = []
    valid_dataloaders = []
    
    for fold in kfold_datasets:
        # Asumiendo que cada fold es un diccionario con 'train' y 'validation' como DataFrames de pandas
        train_dataset, valid_dataset = fold['train'], fold['validation']
        
        # Inicializar NAICSDataset para cada DataFrame
        train_data = NAICSDataset(dataframe=train_dataset, tokenizer=tokenizer, max_len=max_len)
        valid_data = NAICSDataset(dataframe=valid_dataset, tokenizer=tokenizer, max_len=max_len)
        
        # Crear DataLoaders
        train_loader, valid_loader = create_data_loaders_from_datasets(train_data, valid_data, tokenizer, max_len, train_batch_size, valid_batch_size)
        print(train_dataset)
        
        # Agregar a las listas
        train_dataloaders.append(train_loader)
        valid_dataloaders.append(valid_loader)
    
    return train_dataloaders, valid_dataloaders

train_dataloaders, valid_dataloaders = prepare_dataloaders(kfold_2_digits, tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE)
print(train_dataloaders)



Dataset({
    features: ['NAICS', 'BUSINESS_DESCRIPTION', 'label', '__index_level_0__'],
    num_rows: 8032
})
Dataset({
    features: ['NAICS', 'BUSINESS_DESCRIPTION', 'label', '__index_level_0__'],
    num_rows: 8032
})
Dataset({
    features: ['NAICS', 'BUSINESS_DESCRIPTION', 'label', '__index_level_0__'],
    num_rows: 8032
})
[<torch.utils.data.dataloader.DataLoader object at 0x7f89bb37c290>, <torch.utils.data.dataloader.DataLoader object at 0x7f89c0eb4150>, <torch.utils.data.dataloader.DataLoader object at 0x7f89bb4c4c50>]


In [15]:
from transformers import DistilBertModel
import torch

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        # Cargar DistilBERT preentrenado
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        # Una capa lineal que reduce la dimensionalidad de la salida de DistilBERT de 768 a 768.
        # Esto actúa como una capa "pre-clasificadora", pero en este caso, no cambia la dimensión.
        self.pre_classifier = torch.nn.Linear(768, 768)
        # Capa de dropout para regularización, ayudando a prevenir el sobreajuste
        self.dropout = torch.nn.Dropout(0.3)  # Aumento la tasa de dropout para una mayor regularización
        # Capa clasificadora final que mapea de 768 características a 24 categorías (tus etiquetas)
        self.classifier = torch.nn.Linear(768, 24)

    def forward(self, input_ids, attention_mask):
        """
        Forward pass del modelo.
        
        :param input_ids: IDs de tokens para la entrada.
        :param attention_mask: Máscara de atención para evitar que el modelo atienda a tokens de padding.
        """
        # Obtener la salida de DistilBERT. No necesita token_type_ids porque DistilBERT no los usa.
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]  # La salida completa de la secuencia.
        pooler = hidden_state[:, 0]  # Tomamos solo la primera posición de todos los embeddings (representando [CLS])
        pooler = self.pre_classifier(pooler)  # Pasamos por la capa pre-clasificadora
        pooler = torch.nn.Tanh()(pooler)  # Aplicamos la función de activación Tanh
        pooler = self.dropout(pooler)  # Aplicamos dropout para regularización
        output = self.classifier(pooler)  # La salida de la capa clasificadora final
        return output

# Inicialización del modelo
model = DistilBERTClass()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [16]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets) 
loss_fn = torch.nn.BCEWithLogitsLoss()


optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [17]:
from tqdm import tqdm
'''

def train_epoch(model, data_loader, loss_fn, optimizer, device):
    """
    Entrena el modelo para una sola época usando el data_loader proporcionado.
    """
    model.train()  # Pone el modelo en modo entrenamiento
    epoch_loss = 0.0
    num_batches = len(data_loader)
    
    for _, data in tqdm(enumerate(data_loader), desc="Training..."):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.float)
        
        optimizer.zero_grad()  # Limpia los gradientes
        outputs = model(ids, mask)  # Obtiene la salida del modelo
        
        loss = loss_fn(outputs, targets)  # Calcula la pérdida
        loss.backward()  # Retropropaga los errores
        optimizer.step()  # Actualiza los pesos
        
        epoch_loss += loss.item()  # Acumula la pérdida

    average_loss = epoch_loss / num_batches  # Calcula la pérdida promedio
    return average_loss
'''

def train_epoch_v2(model, data_loader, loss_fn, optimizer, device):
    """
    Entrena el modelo para una sola época usando el data_loader proporcionado.
    """
    # Pone el modelo en modo entrenamiento
    model.train()
    epoch_loss = 0.0
    num_batches = 0
    
    # Itera sobre el data_loader
    for batch_idx, batch in enumerate(tqdm(data_loader, desc="Training")):
        try:
            # Asegura que los datos estén en el formato correcto
            if not isinstance(batch, dict):
                raise ValueError("Each batch should be a dictionary.")
            if not all(key in batch for key in ['ids', 'mask', 'targets']):
                raise ValueError("Batch does not contain all required keys.")
            if not all(isinstance(batch[key], torch.Tensor) for key in ['ids', 'mask', 'targets']):
                raise ValueError("Each value in the batch should be a torch.Tensor.")
            
            # Mueve los datos al dispositivo adecuado
            ids = batch['ids'].to(device, dtype=torch.long)
            mask = batch['mask'].to(device, dtype=torch.long)
            targets = batch['targets'].to(device, dtype=torch.float)
            
            # Limpia los gradientes
            optimizer.zero_grad()
            
            # Obtiene la salida del modelo y calcula la pérdida
            outputs = model(ids, mask)
            loss = loss_fn(outputs, targets)
            
            # Retropropaga los errores y actualiza los pesos
            loss.backward()
            optimizer.step()
            
            # Acumula la pérdida y cuenta el número de batches
            epoch_loss += loss.item()
            num_batches += 1
        
        except Exception as e:
            print(f"Error in batch {batch_idx}: {str(e)}")
            continue

    # Calcula la pérdida promedio por batch
    if num_batches == 0:
        raise ValueError("No batches were processed. Check your data_loader and Dataset.")
    
    average_loss = epoch_loss / num_batches
    return average_loss


def train_model(train_dataloaders, model, loss_fn, optimizer, device, epochs=1):
    """
    Entrena el modelo a través de todas las épocas y folds proporcionados.
    """
    model.to(device)  # Mueve el modelo al dispositivo adecuado
    
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        
        for fold, train_loader in enumerate(train_dataloaders, start=1):
            print(f'Starting training for fold {fold}')
            
            average_loss = train_epoch_v2(model, train_loader, loss_fn, optimizer, device)  # Entrena para una época
            
            print(f'Fold: {fold}, Epoch: {epoch+1}, Average Loss: {average_loss:.4f}')

In [18]:
train_model(train_dataloaders, model, loss_fn, optimizer, device, EPOCHS)

Epoch 1/2
Starting training for fold 1


Training:   0%|          | 0/8032 [00:00<?, ?it/s]


ValueError: Index is a list, expected a single integer value.